# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [47]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [48]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [49]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [50]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    
    tokenizer = Tokenizer(num_words=None, filters="", lower=False, split=" ")
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'of': 17, 'won': 19, 'This': 21, 'sentence': 24, 'lazy': 10, '.': 1, 'is': 22, 'jumps': 7, 'The': 4, ',': 14, 'short': 23, 'brown': 5, 'fox': 6, 'By': 12, 'quick': 2, 'lexicography': 18, 'my': 15, 'a': 3, 'Jove': 13, 'over': 8, 'dog': 11, 'the': 9, 'prize': 20, 'study': 16}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [4, 2, 5, 6, 7, 8, 9, 10, 11, 1]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [12, 13, 14, 15, 2, 16, 17, 18, 19, 3, 20, 1]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [21, 22, 3, 23, 24, 1]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [51]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length==None:
        maxLenX = 0
        for sequence in x:
            if len(sequence) > maxLenX:
                maxLenX = len(sequence)
        
        padded = pad_sequences(sequences=x,maxlen=maxLenX, dtype='int32', padding='post', truncating='post', value=0)

    else:
        padded = pad_sequences(sequences=x,maxlen=length, dtype='int32', padding='post', truncating='post', value=0)
    return padded

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 4  2  5  6  7  8  9 10 11  1]
  Output: [ 4  2  5  6  7  8  9 10 11  1  0  0]
Sequence 2 in x
  Input:  [12 13 14 15  2 16 17 18 19  3 20  1]
  Output: [12 13 14 15  2 16 17 18 19  3 20  1]
Sequence 3 in x
  Input:  [21 22  3 23 24  1]
  Output: [21 22  3 23 24  1  0  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [52]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')


Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [53]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [54]:
from keras.layers import GRU, Input, Dense, TimeDistributed, SimpleRNN, LSTM
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 0.01
    hidden_dim = 32
    
    #Build a sequential model
    model = Sequential()
    
    #Adding a GRU layer with 32 hidden unit size. Return sequences = True for many to many output.
    model.add(GRU(hidden_dim, input_shape  = input_shape[1:], return_sequences=True ))
    
    #Adding a dense layer with Softmax activation on french_vocab_size as the final output layer. 
    model.add((Dense(french_vocab_size, activation='softmax')))
    model.summary()
    
    #compile the model with sparse categorical crossentropy as loss function and Adam optimizer.
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=5, validation_split=0.2)

simple_rnn_model.save('saved_models/simple_rnn_model.hdf5')

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 21, 32)            3264      
_________________________________________________________________
dense_17 (Dense)             (None, 21, 344)           11352     
Total params: 14,616
Trainable params: 14,616
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_6 (GRU)                  (None, 23, 32)            3264      
_________________________________________________________________
dense_18 (Dense)             (None, 23, 356)           11748     
Total params: 15,012
Trainable params: 15,012
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/5
110288/

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [62]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # TODO: Build the layers
    learning_rate = 0.01
    embedding_vector_length = 32
    hidden_dim = 32
    
    model = Sequential()
    
    #Adding an embedding layer with embedding vector length = 32, for english_vocab_size
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    
    model.add(GRU(hidden_dim, return_sequences=True ))
    model.add((Dense(french_vocab_size, activation='softmax')))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5, validation_split=0.2)

embed_rnn_model.save('saved_models/embed_rnn_model.hdf5')

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 21, 32)            6368      
_________________________________________________________________
gru_7 (GRU)                  (None, 21, 32)            6240      
_________________________________________________________________
dense_19 (Dense)             (None, 21, 344)           11352     
Total params: 23,960
Trainable params: 23,960
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 23, 32)            7296      
_________________________________________________________________
gru_8 (GRU)                  (None, 23, 32)            6240      
_________________________________________________________________
dense_

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [44]:
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    hidden_dim = 32
    
    model = Sequential()
    
    #Adding a bidirectional LSTM with return_sequences = True
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True),
                        input_shape= input_shape[1:]))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.summary()

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

    
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5, validation_split=0.2)

bd_rnn_model.save('saved_models/bd_rnn_model.hdf5')

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))



(21, 1)
21
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 21, 64)            8704      
_________________________________________________________________
dense_12 (Dense)             (None, 21, 344)           22360     
_________________________________________________________________
activation_3 (Activation)    (None, 21, 344)           0         
Total params: 31,064
Trainable params: 31,064
Non-trainable params: 0
_________________________________________________________________
(23, 1)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 23, 64)            8704      
_________________________________________________________________
dense_13 (Dense)             (None, 23, 356)           23140     
__________________________________________________

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [88]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    learning_rate = 0.01
    hidden_dim = 32
    
    
    model = Sequential()
    
    #Encoder LSTM with return_sequences = false (many to one)
    model.add(LSTM(hidden_dim, input_shape  = input_shape[1:]))
    
    #RepeatVector is used to repeat the output of encoder LSTM 23 (french sequence length) times
    model.add(RepeatVector(output_sequence_length))
    
    #Decoder LSTM with return_sequences = True for many to many
    model.add(LSTM(hidden_dim, return_sequences=True))
    
    #Final dense layer output
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


    return None
tests.test_encdec_model(encdec_model)

# TODO: Train and Print prediction(s)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=5, validation_split=0.2)

encdec_rnn_model.save('saved_models/encdec_rnn_model.hdf5')

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))



output_sequence_length =  21
input shape =  (15, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_103 (LSTM)              (None, 32)                4352      
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 21, 32)            0         
_________________________________________________________________
lstm_104 (LSTM)              (None, 21, 32)            8320      
_________________________________________________________________
dense_52 (Dense)             (None, 21, 344)           11352     
_________________________________________________________________
activation_7 (Activation)    (None, 21, 344)           0         
Total params: 24,024
Trainable params: 24,024
Non-trainable params: 0
_________________________________________________________________
None
output_sequence_length =  23
input shape =  (23, 1)
_____________________________

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [91]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    embedding_vector_length = 128
    hidden_dim = 32
    
    #Layers
    #Input layer : is an embedding with 128 vector length
    #hidden layer : Bidirectional LSTM (return_sequences = Ture)
    #output layer : Dense Softmax 
    
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model = model_final(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

final_rnn_model.save('saved_models/final_rnn_model.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(23,)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_93 (Embedding)     (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 23, 64)            41216     
_________________________________________________________________
dense_55 (Dense)             (None, 23, 356)           23140     
_________________________________________________________________
activation_9 (Activation)    (None, 23, 356)           0         
Total params: 93,540
Trainable params: 93,540
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 1249s - loss: 2.0506 - acc: 0.5509 - val_loss: 1.1071 - val_acc: 0.7018
Epoch 2/10
110288/110288 [==============================] - 393s - loss: 

In [45]:
from keras.layers import Dropout, Embedding

def model_final2(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    hidden_dim = 32
    
    #Layers
    #Input layer : is an embedding with 128 vector length
    #hidden layer : Bidirectional LSTM (return_sequences = Ture)
    #output layer : Dense Softmax 
    
    #Variation
    #Adding a dropout layer
    #Changing batch size to 512
    
    embedding_vector_length = 128
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.summary()
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model2 = model_final2(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model2.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=10, validation_split=0.2)

final_rnn_model2.save('saved_models/final_rnn_model2.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model2.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(23,)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 23, 64)            41216     
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 23, 356)           23140     
_________________________________________________________________
activation_5 (Activation)    (None, 23, 356)           0         
Total params: 93,540
Trainable params: 93,540
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 382s - loss: 1.581

In [93]:
from keras.layers import Dropout

def model_final3(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    embedding_vector_length = 128
    hidden_dim = 128
    
    #variation 
    #Increasing hidden dimention unit size to 128
    #Adding a dropout layer
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model3 = model_final2(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model3.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

final_rnn_model3.save('saved_models/final_rnn_model3.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model3.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(23,)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_95 (Embedding)     (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 23, 64)            41216     
_________________________________________________________________
dense_57 (Dense)             (None, 23, 356)           23140     
_________________________________________________________________
activation_11 (Activation)   (None, 23, 356)           0         
Total params: 93,540
Trainable params: 93,540
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 429s - loss: 2.0866 - acc: 0.5489 - val_loss: 1.1445 - val_acc: 0.6948
Epoch 2/10
110288/110288 [==============================] - 413s - loss: 0

In [94]:
from keras.layers import Dropout

def model_final4(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    embedding_vector_length = 32
    hidden_dim = 32
    
    #Variation
    #Adding 2 Bidirectional LSTM hidden layer
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model4 = model_final2(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model4.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

final_rnn_model4.save('saved_models/final_rnn_model4.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model4.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(23,)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_96 (Embedding)     (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 23, 64)            41216     
_________________________________________________________________
dense_58 (Dense)             (None, 23, 356)           23140     
_________________________________________________________________
activation_12 (Activation)   (None, 23, 356)           0         
Total params: 93,540
Trainable params: 93,540
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 753s - loss: 2.1255 - acc: 0.5375 - val_loss: 1.1618 - val_acc: 0.6921
Epoch 2/10
110288/110288 [==============================] - 409s - loss: 0

In [95]:
from keras.layers import Dropout, Bidirectional, RepeatVector
from keras.layers.embeddings import Embedding


def model_final5(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    #Variation
    #Using encoder decoder along with bidirectional LSTM and Embedding
    
    learning_rate = 0.01
    hidden_dim = 32
    
    embedding_vector_length = 32
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim)))
    model.add(Dropout(0.2))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model5 = model_final5(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

final_rnn_model5.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

final_rnn_model5.save('saved_models/final_rnn_model5.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model5.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(23,)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_97 (Embedding)     (None, 23, 32)            7296      
_________________________________________________________________
bidirectional_7 (Bidirection (None, 64)                16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
repeat_vector_12 (RepeatVect (None, 23, 64)            0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 23, 64)            24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
dense_59 (Dense)             (None, 23, 356)           23140     
_

In [ ]:
#load model for further training. 
from keras.models import load_model

model = load_model('saved_models/final_rnn_model5.hdf5')
model.summary()
model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
model.save('saved_models/final_rnn_model5.hdf5')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_97 (Embedding)     (None, 23, 32)            7296      
_________________________________________________________________
bidirectional_7 (Bidirection (None, 64)                16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
repeat_vector_12 (RepeatVect (None, 23, 64)            0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 23, 64)            24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
dense_59 (Dense)             (None, 23, 356)           23140     
__________

In [78]:
from keras.layers import Dropout

def model_final6(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    hidden_dim = 32
    embedding_vector_length = 128
    
    #Variation - 3 Hidden LSTM Layer and 2 Dense ouput layer with higher Dropout Rate
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model6 = model_final6(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model6.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

final_rnn_model6.save('saved_models/final_rnn_model6.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model6.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(23,)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 23, 64)            41216     
_________________________________________________________________
dropout_7 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 23, 64)            24832     
_________________________________________________________________
dropout_8 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 23, 64)            24832     
_________________________________________________________________
dropout_9 (Dropout)          (None, 23, 64)            0         
_

In [79]:
#load model for further training. 
from keras.models import load_model

model = load_model('saved_models/final_rnn_model6.hdf5')
model.summary()
model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
model.save('saved_models/final_rnn_model6.hdf5')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 23, 64)            41216     
_________________________________________________________________
dropout_7 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 23, 64)            24832     
_________________________________________________________________
dropout_8 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 23, 64)            24832     
_________________________________________________________________
dropout_9 (Dropout)          (None, 23, 64)            0         
__________

In [103]:
from keras.layers import Dropout

def model_final7(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    embedding_vector_length = 128
    hidden_dim = 32
    
    #Variation Encoder Decoder with enbedding, bidirectional LSTM and 2 dense output layer.
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=False)))
    model.add(Dropout(0.2))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Dense(french_vocab_size))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model7 = model_final7(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model7.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

final_rnn_model7.save('saved_models/final_rnn_model7.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model6.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(23,)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_101 (Embedding)    (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 64)                41216     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
repeat_vector_14 (RepeatVect (None, 23, 64)            0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 23, 64)            24832     
_________________________________________________________________
dropout_12 (Dropout)         (None, 23, 64)            0         
_________________________________________________________________
dense_64 (Dense)             (None, 23, 356)           23140     
_

In [46]:
from keras.layers import Dropout

def model_final8(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape[1:])
    print(output_sequence_length)
    
    learning_rate = 0.01
    embedding_vector_length = 128
    hidden_dim = 32
    
    #Variation - 2 layer deep encoder, decoder with 2 dense output layer. 
    
    model = Sequential()
    model.add(Embedding(english_vocab_size, embedding_vector_length, input_shape= input_shape[1:]))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=False)))
    model.add(Dropout(0.2))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=False)))
    model.add(Dropout(0.2))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(french_vocab_size)))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(french_vocab_size))
    model.add(Activation('softmax'))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    print(model.summary())
    return model


#tests.test_model_final(model_final)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
final_rnn_model8 = model_final8(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
final_rnn_model8.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

final_rnn_model8.save('saved_models/final_rnn_model8.hdf5')

# Print prediction(s)
print(logits_to_text(final_rnn_model8.predict(tmp_x[:1])[0], french_tokenizer))

print('Final Model Loaded')

(23,)
23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                41216     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 23, 64)            0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 64)                24832     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 23, 64)            0         
_

#### Model Comparision

Model Name with Description, 	5, 10, 20 epoch validation accuracy, 	Avg Time per Epoch on Laptop CPU (sec).

1. Simple RNN Model	(GRU 32) 61% validation accuracy after 5 epoch. Average time per epoch is 350 Sec. <br /> <br /> 

2. Enbedding with Simple RNN (GRU 32, Embedding 32)	78%	validation accuracy after 5 epoch. Average time per epoch is 350 Sec. <br /> <br /> 

3. Bidirectional RNN	(Bidirectional LSTM 32)	64%	validation accuracy after 5 epoch. Average time per epoch is 320 Sec. <br /> <br /> 

4. Encoder Decoder	(LSTM 32)	53%	validation accuracy after 5 epoch. Average time per epoch is 350 Sec.<br /> <br /> 

5. Combined Model 1	(Bidirectional LSTM 32, Embedding 128)	89% validation accuracy after 5 epoch, 92.5% accuracy after 10 epochs with average run time per epoch 400 Secs.	<br /> <br /> 

6. Combined Model 2	(Combined Model 1, 512 batch size, dropout dense 0.2)	91.40% after 5 epochs	92.95% after 10 epochs with	340 Secs per epoch.	<br /> <br /> 

7. Combined Model 3	(Bidirectional LSTM 32, Embedding 128, Dropout dense 0.2)	89.50% after 5 epochs	92.80% after 10 epochs with	400 Sec per epoch.	<br /> <br /> 

8. Combined Model 4	(2 Bidirection LSTM 32, Embedding 32)	89.10% after 5 epochs	92.70% after 10 epochs	400 Secs per epoch.	<br /> <br /> 

9. Combined Model 5	(encoder decoder along with bidirectional LSTM 32 and Embedding 32)	66.50% after 5 epochs 74% after 10 epochs 80% after 20 epochs with	560 Secs per epoch.<br /> <br /> 

10. <b> Combined Model 6 (3 Hidden LSTM 32 Layer and 2 Dense ouput layer with higher Dropout Rate 0.5, Embedding 128) 	87% after 5 epochs and	93.5% after 10 epochs, 95.5% after 20 epochs with 1000 Sec per epoch. </b> <br /> <br /> 

11. Combined Model 7 (Encoder Decoder with enbedding 128, bidirectional LSTM 32 and 2 dense output layer) has 71.00% after 5 epochs	77.70% after 10 epochs with	880 Sec	per epoch.<br /> <br /> 

12. Combined Model 8 (2 layer deep encoder decoder with 2 dense output layer) has 69.00% after 5 epochs and	74%	after 10 epochs with 850 Sec per epoch. <br /> <br /> 



## Prediction (IMPLEMENTATION)

In [80]:
from keras.models import load_model

loaded_model = load_model('saved_models/final_rnn_model6.hdf5')
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 23, 64)            41216     
_________________________________________________________________
dropout_7 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 23, 64)            24832     
_________________________________________________________________
dropout_8 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 23, 64)            24832     
_________________________________________________________________
dropout_9 (Dropout)          (None, 23, 64)            0         
__________

In [81]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    
    model = load_model('saved_models/final_rnn_model6.hdf5')
    model.summary()
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    
    tmp_sentences = pad(sentences, y.shape[1])
    tmp_sentences = tmp_sentences.reshape((-1, y.shape[-2]))
        
    predictions = model.predict(tmp_sentences, len(tmp_sentences))
        
    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 23, 128)           29184     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 23, 64)            41216     
_________________________________________________________________
dropout_7 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 23, 64)            24832     
_________________________________________________________________
dropout_8 (Dropout)          (None, 23, 64)            0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 23, 64)            24832     
_________________________________________________________________
dropout_9 (Dropout)          (None, 23, 64)            0         
__________

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.